# SU(2) checker

To check for $SU(2)$ symmetry, specify a Hamiltonian as a `SparsePauliOp` in the cariable `h_test`. 

In [1]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp

def has_su2_symmetry(hamiltonian: SparsePauliOp, tol: float = 1e-7):
    """
    Identifies if a Hamiltonian possesses collective SU(2) symmetry.
    """
    num_qubits = hamiltonian.num_qubits
    
    # 1. Construct the total spin generators Jx, Jy, Jz
    # For SU(2) systems, states are represented on a 2D manifold (Bloch sphere) 
    def get_total_spin_op(label):
        ops = []
        for i in range(num_qubits):
            pauli_list = ['I'] * num_qubits
            pauli_list[i] = label
            ops.append("".join(pauli_list))
        # Total spin is the sum of local Pauli matrices 
        return SparsePauliOp.from_list([(op, 0.5) for op in ops])

    generators = {
        'Jx': get_total_spin_op('X'),
        'Jy': get_total_spin_op('Y'),
        'Jz': get_total_spin_op('Z')
    }

    print(f"Checking {num_qubits}-qubit Hamiltonian for SU(2) symmetry...\n")
    
    # 2. Check if [H, J_alpha] = 0 for all alpha 
    is_symmetric = True
    for name, J_alpha in generators.items():
        # Compute commutator: H J - J H
        commutator = (hamiltonian @ J_alpha) - (J_alpha @ hamiltonian)
        commutator = commutator.simplify() # Removes terms with near-zero coefficients

        # 3. Verify if the resulting operator is symmetric
        norm = np.linalg.norm(commutator.coeffs)
        if norm > tol:
            print(f"Symmetry broken for {name}: Commutator norm = {norm:.2e}")
            is_symmetric = False
        else:
            print(f"Symmetry preserved for {name}")

    return is_symmetric

# Test with a Heisenberg Hamiltonian (known SU(2) symmetry)
h_test = SparsePauliOp.from_list([("XX", 1.0), ("YY", 1.0), ("ZZ", 1.0)])

print(f"\nFinal Result: {'Symmetric' if has_su2_symmetry(h_test) else 'Non-symmetric'}")

Checking 2-qubit Hamiltonian for SU(2) symmetry...

Symmetry preserved for Jx
Symmetry preserved for Jy
Symmetry preserved for Jz

Final Result: Symmetric


In [2]:
%pip list | grep qiskit

qiskit                  2.1.2
qiskit-addon-sqd        0.12.0
qiskit-addon-utils      0.3.0
qiskit-aer              0.17.2
qiskit-algorithms       0.4.0
qiskit-ibm-runtime      0.41.1
Note: you may need to restart the kernel to use updated packages.


<div style="background-color:#f0f0f0; padding:10px; border-radius:5px;">
This code is a part of Quantum AI Biomedical Research Lab project 
    
`Estimating molecular ground and excited state energies on quantum computers'
    
© Copyright Renata Wong, 2026.

This code is licensed under the CC BY-NC 4.0 License. You may
obtain a copy of this license in the LICENSE.txt file in the root directory
of this source tree or at https://creativecommons.org/licenses/by-nc/4.0/deed.en.

Any modifications or derivative works of this code must retain this
copyright notice, and modified files need to carry a notice indicating
that they have been altered from the originals.
</div>

<div style="background-color:#f0f0f0; padding:10px; border-radius:5px;">
This work was supported by the National Science and Technology Council (Taiwan) grant No. NSTC 114-2112-M-182-002-MY3 and Chang Gung Memorial Hospital grant No. BMRPL94.
</div>